In [1]:
import torch

BATCH_SIZE = 4 # increase / decrease according to GPU memeory
RESIZE_TO = 512 # resize the image for training and transforms
NUM_EPOCHS = 10 # number of epochs to train for

DEVICE = torch.device('cpu')

# training images and XML files directory
TRAIN_DIR = '../data/rcnn/training'
# validation images and XML files directory
VALID_DIR = '../data/rcnn/validation'

# classes: 0 index is reserved for background
CLASSES = [
    'background', 'abw', 'pbw'
]
NUM_CLASSES = 3

# whether to visualize images after crearing the data loaders
VISUALIZE_TRANSFORMED_IMAGES = False

# location to save model and plots
OUT_DIR = '../results/rcnn'
SAVE_PLOTS_EPOCH = 2 # save loss plots after these many epochs
SAVE_MODEL_EPOCH = 2 # save model after these many epochs

In [2]:
import albumentations as A
import cv2
import numpy as np

from albumentations.pytorch import ToTensorV2

# this class keeps track of the training and validation loss values...
# ... and helps to get the average for each epoch as well
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0
        
    def send(self, value):
        self.current_total += value
        self.iterations += 1
    
    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations
    
    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [3]:
def collate_fn(batch):
    """
    To handle the data loading as different images may have different number 
    of objects and to handle varying size tensors as well.
    """
    return tuple(zip(*batch))

In [4]:
# define the training tranforms
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        A.RandomRotate90(0.5),
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc',
        'label_fields': ['labels']
    })

# define the validation transforms
def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc', 
        'label_fields': ['labels']
    })

# define the negative sample transforms
def get_negative_sample_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ])

In [5]:
def show_tranformed_image(train_loader):
    """
    This function shows the transformed images from the `train_loader`.
    Helps to check whether the tranformed images along with the corresponding
    labels are correct or not.
    Only runs if `VISUALIZE_TRANSFORMED_IMAGES = True` in config.py.
    """
    if len(train_loader) > 0:
        for i in range(1):
            images, targets = next(iter(train_loader))
            images = list(image.to(DEVICE) for image in images)
            targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
            boxes = targets[i]['boxes'].cpu().numpy().astype(np.int32)
            sample = images[i].permute(1, 2, 0).cpu().numpy()
            for box in boxes:
                cv2.rectangle(sample,
                            (box[0], box[1]),
                            (box[2], box[3]),
                            (0, 0, 255), 2)
            cv2.imshow('Transformed image', sample)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

In [6]:
import torch
import cv2
import numpy as np
import os
import glob as glob
import pandas as pd

from torch.utils.data import Dataset, DataLoader

In [7]:
# the dataset class
class WormDataset(Dataset):
    def __init__(self, dir_path, width, height, classes, transforms, negative_sample_transforms):
        self.transforms = transforms
        self.negative_sample_transforms = negative_sample_transforms
        self.dir_path = dir_path
        self.height = height
        self.width = width
        self.classes = classes
        
        # get all the image paths in sorted order
        self.image_paths = glob.glob(f"{self.dir_path}/*[jpg|jpeg]")
        self.all_images = [image_path.split('/')[-1] for image_path in self.image_paths]
        self.all_images = sorted(self.all_images)

    def __getitem__(self, idx):
        # capture the image name and the full image path
        image_name = self.all_images[idx]
        image_path = os.path.join(self.dir_path, image_name)

        # read the image
        image = cv2.imread(image_path)
        # convert BGR to RGB color format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image_resized = cv2.resize(image, (self.width, self.height))
        image_resized /= 255.0
        
        # capture the corresponding XML file for getting the annotations
        annotations_file = image_name.split('.')[0] + '.txt'
        annotations_file_path = os.path.join(self.dir_path, annotations_file)
        
        boxes = []
        labels = []
        
        # get the height and width of the image
        image_width = image.shape[1]
        image_height = image.shape[0]
        
        # box coordinates for xml files are extracted and corrected for image size given

        bounding_boxes_df = pd.read_csv(annotations_file_path, sep='\t', names=["target", 'xmin', 'ymin', 'xmax', 'ymax'])
        bounding_boxes_df['xmin'] = bounding_boxes_df['xmin'] * self.width
        bounding_boxes_df['xmax'] = bounding_boxes_df['xmax'] * self.width
        bounding_boxes_df['ymin'] = bounding_boxes_df['ymin'] * self.height
        bounding_boxes_df['ymax'] = bounding_boxes_df['ymax'] * self.height
        
        # bounding box to tensor
        boxes = bounding_boxes_df[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy()
        labels = bounding_boxes_df['target'].to_numpy()
        
        is_negative_sample = len(boxes) == 0

        if is_negative_sample:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.as_tensor(np.array([0]), dtype=torch.int64)
        else:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            # labels to tensor
            labels = torch.as_tensor(labels, dtype=torch.int64)

        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)

        # prepare the final `target` dictionary
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd
        image_id = torch.tensor([idx])
        target["image_id"] = image_id

        # apply the image transforms
        if not is_negative_sample:
            sample = self.transforms(image = image_resized,
                                     bboxes = target['boxes'],
                                     labels = labels)
            image_resized = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
        elif is_negative_sample:
            sample = self.negative_sample_transforms(image=image_resized,
                                                     labels = labels)
            image_resized = sample['image']
        
        return image_resized, target

    def __len__(self):
        return len(self.all_images)

In [8]:
# prepare the final datasets and data loaders
train_dataset = WormDataset(TRAIN_DIR, RESIZE_TO, RESIZE_TO, CLASSES, get_train_transform(), get_negative_sample_transform())
valid_dataset = WormDataset(VALID_DIR, RESIZE_TO, RESIZE_TO, CLASSES, get_valid_transform(), get_negative_sample_transform())
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(valid_dataset)}\n")

Number of training samples: 8763
Number of validation samples: 974



In [9]:
# # sanity check of the Dataset pipeline with sample visualization
# dataset = WormDataset(
#     TRAIN_DIR, RESIZE_TO, RESIZE_TO, CLASSES
# )
# print(f"Number of training images: {len(dataset)}")

# # function to visualize a single sample
# def visualize_sample(image, target):
#     box = target['boxes'][0] if len(target['boxes']) > 0 else []
#     label = CLASSES[target['labels'][0]] if len(target['labels']) > 0 else []
#     if len(box) != 0:
#         cv2.rectangle(
#             image, 
#             (int(box[0]), int(box[1])), (int(box[2]), int(box[3])),
#             (0, 255, 0), 1
#         )
#         cv2.putText(
#             image, label, (int(box[0]), int(box[1]-5)), 
#             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2
#         )
#     cv2.imshow('Image', image)
#     cv2.waitKey(0)
    
# NUM_SAMPLES_TO_VISUALIZE = 5
# for i in range(NUM_SAMPLES_TO_VISUALIZE):
#     image, target = dataset[i]
#     visualize_sample(image, target)

In [10]:
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN_ResNet50_FPN_V2_Weights

def create_model(num_classes):
    
    # load Faster RCNN pre-trained model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT)
    
    # get the number of input features 
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # define a new head for the detector with required number of classes
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 

    return model

In [11]:
from tqdm.auto import tqdm

import torch
import matplotlib.pyplot as plt
import time

plt.style.use('ggplot')

In [12]:
# function for running training iterations
def train(train_data_loader, model, optimizer, train_loss_hist, train_loss_list):
    print('Training')
    
    for i, data in enumerate(train_data_loader):
        optimizer.zero_grad()
        images, targets = data
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        train_loss_list.append(loss_value)

        train_loss_hist.send(loss_value)

        losses.backward()
        optimizer.step()
    
        # update the loss value beside the progress bar for each iteration
        print(f"Loss: {loss_value:.4f} ,Index: {i}, Total: {len(train_data_loader)}")
    return train_loss_list

In [13]:
# function for running validation iterations
def validate(valid_data_loader, model, val_loss_hist, val_loss_list):
    print('Validating')
    
    for i, data in enumerate(valid_data_loader):
        images, targets = data
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
        
        with torch.no_grad():
            loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        val_loss_list.append(loss_value)

        val_loss_hist.send(loss_value)

        # update the loss value beside the progress bar for each iteration
        print(f"Loss: {loss_value:.4f} ,Index: {i}, Total: {len(valid_data_loader)}")
    return val_loss_list

In [14]:
# initialize the model and move to the computation device
model = create_model(num_classes=NUM_CLASSES)
model = model.to(DEVICE)
# get the model parameters
params = [p for p in model.parameters() if p.requires_grad]
# define the optimizer
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)

# initialize the Averager class
train_loss_hist = Averager()
val_loss_hist = Averager()
# train and validation loss lists to store loss values of all...
# ... iterations till ena and plot graphs for all iterations
train_loss_list = []
val_loss_list = []

all_training_losses = []
all_validation_losses = []

# name to save the trained model with
MODEL_NAME = 'rcnn'

# whether to show transformed images from data loader or not
if VISUALIZE_TRANSFORMED_IMAGES:
    show_tranformed_image(train_loader)

# start the training epochs
for epoch in range(NUM_EPOCHS):
    print(f"\nEPOCH {epoch+1} of {NUM_EPOCHS}")

    # reset the training and validation loss histories for the current epoch
    train_loss_hist.reset()
    val_loss_hist.reset()

    # create two subplots, one for each, training and validation
    figure_1, train_ax = plt.subplots()
    figure_2, valid_ax = plt.subplots()

    # start timer and carry out training and validation
    start = time.time()
    train_loss = train(train_loader, model, optimizer, train_loss_hist, train_loss_list)
    val_loss = validate(valid_loader, model, val_loss_hist, val_loss_list)
    print(f"Epoch #{epoch} train loss: {train_loss_hist.value:.3f}")   
    print(f"Epoch #{epoch} validation loss: {val_loss_hist.value:.3f}")   
    end = time.time()
    print(f"Took {((end - start) / 60):.3f} minutes for epoch {epoch}")

    if (epoch+1) % SAVE_MODEL_EPOCH == 0: # save model after every n epochs
        torch.save(model.state_dict(), f"{OUT_DIR}/model{epoch+1}.pth")
        print('SAVING MODEL COMPLETE...\n')
    
    if (epoch+1) % SAVE_PLOTS_EPOCH == 0: # save loss plots after n epochs
        train_ax.plot(train_loss, color='blue')
        train_ax.set_xlabel('iterations')
        train_ax.set_ylabel('train loss')
        valid_ax.plot(val_loss, color='red')
        valid_ax.set_xlabel('iterations')
        valid_ax.set_ylabel('validation loss')
        figure_1.savefig(f"{OUT_DIR}/train_loss_{epoch+1}.png")
        figure_2.savefig(f"{OUT_DIR}/valid_loss_{epoch+1}.png")
        print('SAVING PLOTS COMPLETE...')
    
    if (epoch+1) == NUM_EPOCHS: # save loss plots and model once at the end
        train_ax.plot(train_loss, color='blue')
        train_ax.set_xlabel('iterations')
        train_ax.set_ylabel('train loss')
        valid_ax.plot(val_loss, color='red')
        valid_ax.set_xlabel('iterations')
        valid_ax.set_ylabel('validation loss')
        figure_1.savefig(f"{OUT_DIR}/train_loss_{epoch+1}.png")
        figure_2.savefig(f"{OUT_DIR}/valid_loss_{epoch+1}.png")

        torch.save(model.state_dict(), f"{OUT_DIR}/model{epoch+1}.pth")
    
    all_training_losses.append(train_loss_hist.value)
    all_validation_losses.append(val_loss_hist.value)
    
    plt.close('all')


EPOCH 1 of 10
Training
Loss: 1.3022 ,Index: 0, Total: 2191
Loss: 2.6468 ,Index: 1, Total: 2191
Loss: 2.0486 ,Index: 2, Total: 2191
Loss: 0.9286 ,Index: 3, Total: 2191
Loss: 1.7426 ,Index: 4, Total: 2191
Loss: 0.6839 ,Index: 5, Total: 2191
Loss: 1.4379 ,Index: 6, Total: 2191
Loss: 1.4712 ,Index: 7, Total: 2191
Loss: 1.5023 ,Index: 8, Total: 2191
Loss: 0.7829 ,Index: 9, Total: 2191
Loss: 0.3054 ,Index: 10, Total: 2191
Loss: 0.6782 ,Index: 11, Total: 2191
Loss: 0.1923 ,Index: 12, Total: 2191
Loss: 0.6364 ,Index: 13, Total: 2191
Loss: 1.5184 ,Index: 14, Total: 2191
Loss: 1.4022 ,Index: 15, Total: 2191
Loss: 0.8770 ,Index: 16, Total: 2191
Loss: 1.5587 ,Index: 17, Total: 2191
Loss: 0.5387 ,Index: 18, Total: 2191
Loss: 0.8423 ,Index: 19, Total: 2191
Loss: 0.3650 ,Index: 20, Total: 2191
Loss: 0.9353 ,Index: 21, Total: 2191
Loss: 1.7754 ,Index: 22, Total: 2191
Loss: 1.1578 ,Index: 23, Total: 2191
Loss: 0.5839 ,Index: 24, Total: 2191
Loss: 0.8801 ,Index: 25, Total: 2191
Loss: 0.4889 ,Index: 26,

Invalid SOS parameters for sequential JPEG


Loss: 0.2315 ,Index: 1081, Total: 2191
Loss: 0.4365 ,Index: 1082, Total: 2191
Loss: 0.5875 ,Index: 1083, Total: 2191
Loss: 0.1908 ,Index: 1084, Total: 2191
Loss: 0.6027 ,Index: 1085, Total: 2191
Loss: 0.4192 ,Index: 1086, Total: 2191
Loss: 0.2776 ,Index: 1087, Total: 2191
Loss: 0.5403 ,Index: 1088, Total: 2191
Loss: 0.2136 ,Index: 1089, Total: 2191
Loss: 0.4504 ,Index: 1090, Total: 2191
Loss: 0.8855 ,Index: 1091, Total: 2191
Loss: 0.2668 ,Index: 1092, Total: 2191
Loss: 0.7908 ,Index: 1093, Total: 2191
Loss: 0.2943 ,Index: 1094, Total: 2191
Loss: 0.7608 ,Index: 1095, Total: 2191
Loss: 0.5137 ,Index: 1096, Total: 2191
Loss: 0.2629 ,Index: 1097, Total: 2191
Loss: 0.6433 ,Index: 1098, Total: 2191
Loss: 0.2652 ,Index: 1099, Total: 2191
Loss: 0.5229 ,Index: 1100, Total: 2191
Loss: 0.4901 ,Index: 1101, Total: 2191
Loss: 0.3616 ,Index: 1102, Total: 2191
Loss: 0.2657 ,Index: 1103, Total: 2191
Loss: 0.1839 ,Index: 1104, Total: 2191
Loss: 0.6934 ,Index: 1105, Total: 2191
Loss: 0.1936 ,Index: 1106

Invalid SOS parameters for sequential JPEG


Loss: 0.2303 ,Index: 1264, Total: 2191
Loss: 0.4921 ,Index: 1265, Total: 2191
Loss: 0.5233 ,Index: 1266, Total: 2191
Loss: 0.8459 ,Index: 1267, Total: 2191
Loss: 0.4218 ,Index: 1268, Total: 2191
Loss: 0.1346 ,Index: 1269, Total: 2191
Loss: 0.5821 ,Index: 1270, Total: 2191
Loss: 0.5080 ,Index: 1271, Total: 2191
Loss: 0.5044 ,Index: 1272, Total: 2191
Loss: 0.3657 ,Index: 1273, Total: 2191
Loss: 0.1147 ,Index: 1274, Total: 2191
Loss: 0.4388 ,Index: 1275, Total: 2191
Loss: 0.2736 ,Index: 1276, Total: 2191
Loss: 0.8902 ,Index: 1277, Total: 2191
Loss: 0.4851 ,Index: 1278, Total: 2191
Loss: 0.2799 ,Index: 1279, Total: 2191
Loss: 0.1217 ,Index: 1280, Total: 2191
Loss: 0.8082 ,Index: 1281, Total: 2191
Loss: 0.2465 ,Index: 1282, Total: 2191
Loss: 0.7643 ,Index: 1283, Total: 2191
Loss: 0.2526 ,Index: 1284, Total: 2191
Loss: 0.4210 ,Index: 1285, Total: 2191
Loss: 0.1576 ,Index: 1286, Total: 2191
Loss: 0.4309 ,Index: 1287, Total: 2191
Loss: 0.4784 ,Index: 1288, Total: 2191
Loss: 0.2333 ,Index: 1289

...

Loss: 0.0807 ,Index: 1595, Total: 2191
Loss: 0.1868 ,Index: 1596, Total: 2191
Loss: 0.2706 ,Index: 1597, Total: 2191
Loss: 0.2927 ,Index: 1598, Total: 2191
Loss: 0.1617 ,Index: 1599, Total: 2191
Loss: 0.4946 ,Index: 1600, Total: 2191
Loss: 0.1837 ,Index: 1601, Total: 2191
Loss: 0.3341 ,Index: 1602, Total: 2191
Loss: 0.1742 ,Index: 1603, Total: 2191
Loss: 0.2648 ,Index: 1604, Total: 2191
Loss: 0.3737 ,Index: 1605, Total: 2191
Loss: 0.3684 ,Index: 1606, Total: 2191
Loss: 0.2447 ,Index: 1607, Total: 2191
Loss: 0.1126 ,Index: 1608, Total: 2191
Loss: 0.4518 ,Index: 1609, Total: 2191
Loss: 0.7540 ,Index: 1610, Total: 2191
Loss: 0.6708 ,Index: 1611, Total: 2191
Loss: 0.7490 ,Index: 1612, Total: 2191
Loss: 0.2415 ,Index: 1613, Total: 2191
Loss: 0.1177 ,Index: 1614, Total: 2191
Loss: 0.4264 ,Index: 1615, Total: 2191
Loss: 0.2555 ,Index: 1616, Total: 2191
Loss: 0.5612 ,Index: 1617, Total: 2191
Loss: 0.1542 ,Index: 1618, Total: 2191
Loss: 0.4404 ,Index: 1619, Total: 2191
Loss: 0.2108 ,Index: 1620

Invalid SOS parameters for sequential JPEG


Loss: 0.1228 ,Index: 2085, Total: 2191
Loss: 0.3209 ,Index: 2086, Total: 2191
Loss: 0.4545 ,Index: 2087, Total: 2191
Loss: 0.2294 ,Index: 2088, Total: 2191
Loss: 0.2457 ,Index: 2089, Total: 2191
Loss: 0.3311 ,Index: 2090, Total: 2191
Loss: 0.4050 ,Index: 2091, Total: 2191
Loss: 0.5172 ,Index: 2092, Total: 2191
Loss: 0.1251 ,Index: 2093, Total: 2191
Loss: 0.3794 ,Index: 2094, Total: 2191
Loss: 0.2900 ,Index: 2095, Total: 2191
Loss: 0.1282 ,Index: 2096, Total: 2191
Loss: 0.0845 ,Index: 2097, Total: 2191
Loss: 0.1933 ,Index: 2098, Total: 2191
Loss: 0.2004 ,Index: 2099, Total: 2191
Loss: 1.1333 ,Index: 2100, Total: 2191
Loss: 0.1511 ,Index: 2101, Total: 2191
Loss: 0.5009 ,Index: 2102, Total: 2191
Loss: 0.3156 ,Index: 2103, Total: 2191
Loss: 0.2311 ,Index: 2104, Total: 2191
Loss: 0.2223 ,Index: 2105, Total: 2191
Loss: 0.5053 ,Index: 2106, Total: 2191
Loss: 0.4085 ,Index: 2107, Total: 2191
Loss: 0.3160 ,Index: 2108, Total: 2191
Loss: 0.3938 ,Index: 2109, Total: 2191
Loss: 0.1492 ,Index: 2110

Invalid SOS parameters for sequential JPEG


Loss: 0.3792 ,Index: 22, Total: 244
Loss: 0.1345 ,Index: 23, Total: 244
Loss: 0.3274 ,Index: 24, Total: 244
Loss: 0.2563 ,Index: 25, Total: 244
Loss: 0.1132 ,Index: 26, Total: 244
Loss: 0.4839 ,Index: 27, Total: 244
Loss: 0.2652 ,Index: 28, Total: 244
Loss: 0.2547 ,Index: 29, Total: 244
Loss: 0.2098 ,Index: 30, Total: 244
Loss: 0.2401 ,Index: 31, Total: 244
Loss: 0.3184 ,Index: 32, Total: 244
Loss: 0.2002 ,Index: 33, Total: 244
Loss: 0.1587 ,Index: 34, Total: 244
Loss: 0.6542 ,Index: 35, Total: 244
Loss: 0.2771 ,Index: 36, Total: 244
Loss: 0.1061 ,Index: 37, Total: 244
Loss: 0.2308 ,Index: 38, Total: 244
Loss: 0.5587 ,Index: 39, Total: 244
Loss: 0.2816 ,Index: 40, Total: 244
Loss: 0.3728 ,Index: 41, Total: 244
Loss: 0.1703 ,Index: 42, Total: 244
Loss: 0.3116 ,Index: 43, Total: 244
Loss: 0.3408 ,Index: 44, Total: 244
Loss: 0.6776 ,Index: 45, Total: 244
Loss: 0.3893 ,Index: 46, Total: 244
Loss: 0.2407 ,Index: 47, Total: 244
Loss: 0.6163 ,Index: 48, Total: 244
Loss: 0.3916 ,Index: 49, Tot

In [24]:
train_val_df = pd.DataFrame({
    'epochs': np.arange(1, 11, 1, dtype=int),
    'training_loss': all_training_losses,
    'validation_loss': all_validation_losses,
})
train_val_df.to_csv('../results/rcnn/train_val_results.csv', index=False)
train_val_df

,epochs,training_loss,validation_loss
0,1,0.462712,0.389970
1,2,0.396065,0.370712
2,3,0.380058,0.359849
3,4,0.372427,0.350515
4,5,0.363129,0.353558
5,6,0.359340,0.343766
6,7,0.356544,0.343392
7,8,0.352014,0.334505
8,9,0.347437,0.336476
9,10,0.346282,0.335564


In [16]:
import numpy as np
import cv2
import torch
import glob as glob

# set the computation device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# load the model and the trained weights
model = create_model(num_classes=NUM_CLASSES).to(device)
model.load_state_dict(torch.load(
    f'{OUT_DIR}/model10.pth', map_location=device
))
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [17]:
# directory where all the images are present
DIR_TEST = '../data/rcnn/testing/'
test_images = glob.glob(f"{DIR_TEST}/*")
print(f"Test instances: {len(test_images)}")

# define the detection threshold...
# ... any detection having score below this will be discarded
detection_threshold = 0.5

Test instances: 2803


In [18]:
results = []
for i in range(len(test_images)):
    # get the image file name for saving output later on
    image_name = test_images[i].split('/')[-1].split('.')[0]
    image = cv2.imread(test_images[i])
    orig_image = image.copy()
    # BGR to RGB
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
    # make the pixel range between 0 and 1
    image /= 255.0
    # bring color channels to front
    image = np.transpose(image, (2, 0, 1)).astype(np.float)
    # convert to tensor
    image = torch.tensor(image, dtype=torch.float)
    # add batch dimension
    image = torch.unsqueeze(image, 0)
    with torch.no_grad():
        outputs = model(image)
    
    # load all detection to CPU for further operations
    outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]
    # carry further only if there are detected boxes
    if len(outputs[0]['boxes']) != 0:
        boxes = outputs[0]['boxes'].data.numpy()
        scores = outputs[0]['scores'].data.numpy()
        # filter out boxes according to `detection_threshold`
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        draw_boxes = boxes.copy()
        # get all the predicted class names
        pred_classes = [CLASSES[i] for i in outputs[0]['labels'].numpy()]
        
        # draw the bounding boxes and write the class name on top of it
        for j, box in enumerate(draw_boxes):
            cv2.rectangle(orig_image,
                        (int(box[0]), int(box[1])),
                        (int(box[2]), int(box[3])),
                        (0, 0, 255), 2)
            cv2.putText(orig_image, pred_classes[j], 
                        (int(box[0]), int(box[1]-5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 
                        2, lineType=cv2.LINE_AA)
            results.append({
                "image": test_images[i].split('/')[-1],
                "bounding_box": box,
                "prediction": pred_classes[j]
            })

        # cv2.imshow('Prediction', orig_image)
        # cv2.waitKey(1)
        cv2.imwrite(f"{OUT_DIR}/test_predictions/{image_name}.jpg", orig_image,)
    print(f"Image {i+1} done...")
    print('-'*50)

/var/folders/n2/bwdv1zhn55ngx2l75t_7jwkw0000gn/T/ipykernel_10737/1240347181.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  image = np.transpose(image, (2, 0, 1)).astype(np.float)


Image 1 done...
--------------------------------------------------
Image 2 done...
--------------------------------------------------
Image 3 done...
--------------------------------------------------
Image 4 done...
--------------------------------------------------
Image 5 done...
--------------------------------------------------
Image 6 done...
--------------------------------------------------
Image 7 done...
--------------------------------------------------
Image 8 done...
--------------------------------------------------
Image 9 done...
--------------------------------------------------
Image 10 done...
--------------------------------------------------
Image 11 done...
--------------------------------------------------
Image 12 done...
--------------------------------------------------
Image 13 done...
--------------------------------------------------
Image 14 done...
--------------------------------------------------
Image 15 done...
----------------------------------------

Invalid SOS parameters for sequential JPEG


Image 928 done...
--------------------------------------------------
Image 929 done...
--------------------------------------------------
Image 930 done...
--------------------------------------------------
Image 931 done...
--------------------------------------------------
Image 932 done...
--------------------------------------------------
Image 933 done...
--------------------------------------------------
Image 934 done...
--------------------------------------------------
Image 935 done...
--------------------------------------------------
Image 936 done...
--------------------------------------------------
Image 937 done...
--------------------------------------------------
Image 938 done...
--------------------------------------------------
Image 939 done...
--------------------------------------------------
Image 940 done...
--------------------------------------------------
Image 941 done...
--------------------------------------------------
Image 942 done...
----------------

In [19]:
import pandas as pd

rcnn_results_df = pd.DataFrame(results)
rcnn_results_df

,image,bounding_box,prediction
0,id_3b59b5876e0a54fd5fd38188.jpg,"[381, 553, 405, 580]",pbw
1,id_3b59b5876e0a54fd5fd38188.jpg,"[409, 205, 434, 236]",pbw
2,id_3b59b5876e0a54fd5fd38188.jpg,"[608, 468, 631, 501]",pbw
3,id_3b59b5876e0a54fd5fd38188.jpg,"[744, 744, 772, 771]",pbw
4,id_3b59b5876e0a54fd5fd38188.jpg,"[587, 330, 609, 360]",pbw
...,...,...,...
39648,id_cbb6c9e7d0c2981f3932a3e1.jpg,"[1485, 211, 1886, 848]",abw
39649,id_cbb6c9e7d0c2981f3932a3e1.jpg,"[451, 43, 843, 782]",abw
39650,id_cbb6c9e7d0c2981f3932a3e1.jpg,"[1296, 1625, 1806, 2438]",abw
39651,id_cbb6c9e7d0c2981f3932a3e1.jpg,"[2283, 2133, 2924, 2990]",abw


In [20]:
rcnn_predictions_df = rcnn_results_df.groupby(['image', 'prediction']).size().reset_index(name='target')
rcnn_predictions_df['image'] = rcnn_predictions_df['image'].apply(lambda x: os.path.splitext(x)[0])
rcnn_predictions_df['Image_ID'] = (rcnn_predictions_df['image'] + "_" + rcnn_predictions_df['prediction']).str.lower()
rcnn_predictions_df = rcnn_predictions_df.drop(columns=['image', 'prediction'])
rcnn_predictions_df

,target,Image_ID
0,4,id_00332970f80fa9a47a39516d_abw
1,22,id_0035981bc3ae42eb5b57a317_pbw
2,9,id_005102f664b820f778291dee_abw
3,5,id_0066456f5fb2cd858c69ab39_abw
4,2,id_00887bebda26184c36e18e00_pbw
...,...,...
2389,11,id_ff40ad3bb245a697d799eeef_abw
2390,1,id_ff6b60935a327d1c2779bc5e_pbw
2391,8,id_ffad8f3773a4222f8fe5ba1a_pbw
2392,100,id_ffb65e6de900c49d8f2ef95a_pbw


In [21]:
test_df = pd.read_csv('../data/test.csv')

abw_df = test_df.copy(deep=True)
abw_df['worm_type'] = 'abw'
pbw_df = test_df.copy(deep=True)
pbw_df['worm_type'] = 'pbw'

full_test_df = pd.concat([abw_df, pbw_df], ignore_index=True)
full_test_df['image_id_worm'] = full_test_df['image_id_worm'].apply(lambda x: os.path.splitext(x)[0])
full_test_df['image_id_worm'] = (full_test_df['image_id_worm'] + "_" + full_test_df['worm_type']).str.lower()
full_test_df = full_test_df.drop(columns=['worm_type'])
full_test_df = full_test_df.rename(columns={"image_id_worm": "Image_ID"})
full_test_df

,Image_ID
0,id_00332970f80fa9a47a39516d_abw
1,id_0035981bc3ae42eb5b57a317_abw
2,id_005102f664b820f778291dee_abw
3,id_0066456f5fb2cd858c69ab39_abw
4,id_007159c1fa015ba6f394deeb_abw
...,...
5601,id_ffad8f3773a4222f8fe5ba1a_pbw
5602,id_ffb65e6de900c49d8f2ef95a_pbw
5603,id_ffbcb27fa549278f47505515_pbw
5604,id_ffc0e41e10b0c964d4a02811_pbw


In [22]:
final_testing_df = rcnn_predictions_df.copy(deep=True)

final_testing_df = pd.merge(full_test_df, final_testing_df, on="Image_ID", how="left")

final_testing_df['target'] = final_testing_df['target'].fillna(0)
final_testing_df['target'] = pd.to_numeric(final_testing_df['target'], downcast='integer')
final_testing_df = final_testing_df.sort_values(by='Image_ID').reset_index(drop=True)

final_testing_df.to_csv('../results/rcnn/rcnn_test_results_2.csv', index=False)
final_testing_df

,Image_ID,target
0,id_00332970f80fa9a47a39516d_abw,4
1,id_00332970f80fa9a47a39516d_pbw,0
2,id_0035981bc3ae42eb5b57a317_abw,0
3,id_0035981bc3ae42eb5b57a317_pbw,22
4,id_005102f664b820f778291dee_abw,9
...,...,...
5601,id_ffbcb27fa549278f47505515_pbw,0
5602,id_ffc0e41e10b0c964d4a02811_abw,0
5603,id_ffc0e41e10b0c964d4a02811_pbw,0
5604,id_fff8c253115aacded09ad7ed_abw,0
